# RDF und SPARQL Sandkasten

In [ ]:
# nötige Module importieren
%pip install -q ipywidgets==8.1.1 ipycytoscape networkx rdflib
import rdflib
import ipycytoscape as cy
import networkx as nx
import pandas as pd

## Funktionsdefinitionen

In [ ]:
# Funktionsfür die Konvertierung von RDF zu NetworkX
def rdflib_to_networkx(rdf_graph):
    G = nx.DiGraph()  # Use Graph() if you want an undirected graph
    for subj, pred, obj in rdf_graph:
        if isinstance(subj, rdflib.URIRef):
            subj = rdf_graph.qname(subj)
        elif isinstance(subj, rdflib.BNode):
            subj = f"_:bnode_{subj}"  # Convert blank node to a unique string
        pred = rdf_graph.qname(pred)
        if isinstance(obj, rdflib.URIRef):
            obj = rdf_graph.qname(obj)
        elif isinstance(obj, rdflib.BNode):
            obj = f"_:bnode_{obj}"  # Convert blank node to a unique string
        elif isinstance(obj, rdflib.Literal):
            obj = str(obj)  # Convert Literal to its string representation
        G.add_edge(str(subj), str(obj), label=str(pred))
    return G

In [ ]:
# Funktionsdefinition, um einen TTL String zu parsen und als Graph zu plotten
def parse_and_plot(ttl_string):
    g = rdflib.Graph()
    g.parse(data = ttl_string)
    nx_graph = rdflib_to_networkx(g)
    plot = cy.CytoscapeWidget()
    plot.graph.add_graph_from_networkx(nx_graph, directed=True)

    style = [
        {
            'selector': 'node',
             'style': {
                'font-family': 'helvetica',
                'font-size': '12px',
                 'color': 'white',
                'text-outline-width': 2,
                'text-outline-color': 'green',
                'background-color': 'green',
                'content': 'data(id)',
                'text-valign': 'center',
             }
        },
        {
            'selector': 'edge.directed',
            'style': {
                'font-family': 'helvetica',
                'font-size': '12px',
                'label': 'data(label)',
                'color': 'white',
                'text-outline-width': 2,
                'text-outline-color': 'orange',
                'background-color': 'orange',
                'curve-style': 'bezier',
                'target-arrow-shape': 'triangle',
            }
        }
    ]
    
    plot.set_style(style)
    return plot

In [ ]:
# Funktionsdefinition, um eine Query gegen eine lokale Turtle Datei auszuführen
def query(ttl_string, query_string):
    
    g = rdflib.Graph()
    g.parse(data = ttl_string)

    qres = g.query(query_string)

    df = pd.DataFrame(qres, columns=qres.vars)
    return df

## Spezifikationen

Diese technischen Dokumente sind sehr lesenwert (und für Spezifikationen auch sehr leserlich geschrieben):

- [RDF Primer](https://www.w3.org/TR/rdf11-primer/)
- [Turtle](https://www.w3.org/TR/turtle/)
- [SPARQL](https://www.w3.org/TR/sparql11-query/)

## Basis Turtle

Bilde folgende Informationen als RDF Triple in Turtle Serialisierung ab:

- Karin Keller Sutter ist eine FDP Bundesrätin
- Viola Amherd ist eine Bundesrätin in der Partei "die Mitte"
- NadineMasshardt ist eine SP Nationalrätin, die in Affoltern am Albis geboren wurde
- Maya Graf ist eine Ständerätin der Grünen und ihr Vater ist Fritz Graf
- Fritz Graf ist im Baselbieter Landrat für die SVP und Maya Graf seine Tochter
- FDP, die Mitte, SP, Grüne und SVP sind politische Parteien 

Nutze für die URIs `<https://example.com/...>`

In [ ]:
ttl_string = """

<https://example.com/KarinKeller-Sutter> <https://example.com/position> <https://example.com/Bundesrat>.
<https://example.com/KarinKeller-Sutter> <https://example.com/partei> <https://example.com/FDP>.

<https://example.com/ViolaAmherd> <https://example.com/position> <https://example.com/Bundesrat>.
<https://example.com/ViolaAmherd> <https://example.com/partei> <https://example.com/DieMitte>.

<https://example.com/NadineMasshardt> <https://example.com/position> <https://example.com/Nationalrat>.
<https://example.com/NadineMasshardt> <https://example.com/partei> <https://example.com/SP>.
<https://example.com/NadineMasshardt> <https://example.com/geborenIn> <https://example.com/AffolternAmAlbis>.

<https://example.com/MayaGraf> <https://example.com/position> <https://example.com/Staenderat>.
<https://example.com/MayaGraf> <https://example.com/partei> <https://example.com/Gruene>.
<https://example.com/MayaGraf> <https://example.com/vater> <https://example.com/FritzGraf>.

<https://example.com/FritzGraf> <https://example.com/position> <https://example.com/BaselbieterLandrat>.
<https://example.com/FritzGraf> <https://example.com/partei> <https://example.com/SVP>.
<https://example.com/FritzGraf> <https://example.com/kind> <https://example.com/MayaGraf>.

<https://example.com/FDP> <https://example.com/ist> <https://example.com/Partei>.
<https://example.com/DieMitte> <https://example.com/ist> <https://example.com/Partei>.
<https://example.com/SP> <https://example.com/ist> <https://example.com/Partei>.
<https://example.com/Gruene> <https://example.com/ist> <https://example.com/Partei>.
<https://example.com/SVP> <https://example.com/ist> <https://example.com/Partei>.

"""

plot = parse_and_plot(ttl_string)
display(plot)

## TTL mit Prefix

Ersetze in Deinen Triples `<https://example.com/>` durch einen Prefix (am besten den Empty Prefix; [Base vs. Empty Prefix](https://stackoverflow.com/questions/34146707/turtle-difference-between-base-and-empty-prefix))

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

:KarinKeller-Sutter :position :Bundesrat.
:KarinKeller-Sutter :partei :FDP.

:ViolaAmherd :position :Bundesrat.
:ViolaAmherd :partei :DieMitte.

:NadineMasshardt :position :Nationalrat.
:NadineMasshardt :partei :SP.
:NadineMasshardt :geborenIn :AffolternAmAlbis.

:MayaGraf :position :Staenderat.
:MayaGraf :partei :Gruene.
:MayaGraf :vater :FritzGraf.

:FritzGraf :position :BaselbieterLandrat.
:FritzGraf :partei :SVP.
:FritzGraf :kind :MayaGraf.

:FDP :ist :Partei.
:DieMitte :ist :Partei.
:SP :ist :Partei.
:Gruene :ist :Partei.
:SVP :ist :Partei.

"""

## TTL ohne Wiederholungen

Ersetze doppelte Subjekte mit Hilfe von [Prädikat-Listen](https://www.w3.org/TR/turtle/#predicate-lists).

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.

:KarinKeller-Sutter :position :Bundesrat;
    :partei :FDP.

:ViolaAmherd :position :Bundesrat;
    :partei :DieMitte.

:NadineMasshardt :position :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis.

:MayaGraf :position :Staenderat;
    :partei :Gruene;
    :vater :FritzGraf.

:FritzGraf :position :BaselbieterLandrat;
    :partei :SVP;
    :kind :MayaGraf.

:FDP :ist :Partei.
:DieMitte :ist :Partei.
:SP :ist :Partei.
:Gruene :ist :Partei.
:SVP :ist :Partei.

"""

## TTL mit externem Vokabular


In [ ]:
ttl_string = """

@prefix : <https://example.com/>.
@prefix schema: <http://schema.org/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.

:KarinKeller-Sutter schema:hasOccupation :Bundesrat;
    :partei :FDP.

:ViolaAmherd schema:hasOccupation :Bundesrat;
    :partei :DieMitte.

:NadineMasshardt schema:hasOccupation :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis.

:MayaGraf schema:hasOccupation :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf schema:hasOccupation :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.

:FDP rdf:type :Partei.
:DieMitte rdf:type :Partei.
:SP rdf:type :Partei.
:Gruene rdf:type :Partei.
:SVP rdf:type :Partei.

"""

## TTL mit Literals

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.

:KarinKeller-Sutter schema:hasOccupation :Bundesrat;
    :partei :FDP;
    :anzahlGeschwister 3.

:ViolaAmherd schema:hasOccupation :Bundesrat;
    :partei :DieMitte;
    :vollerName "Viola Patricia Amherd".

:NadineMasshardt schema:hasOccupation :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis;
    schema:birthDate "1984-10-04"^^xsd:date.

:MayaGraf schema:hasOccupation :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf schema:hasOccupation :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.

:FDP rdf:type :Partei.
:DieMitte rdf:type :Partei.
:SP rdf:type :Partei.
:Gruene rdf:type :Partei.
:SVP rdf:type :Partei.

"""

## TTL mit Blank Nodes

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.


:Zuerich :population :zh_pop.
:zh_pop :number 423193;
    :valid "2021-12-31"^^xsd:date.

:Bern :population [
    :number 134290;
    :valid "2021-12-31"^^xsd:date ].

"""

## SPARQL Query: Alle Triples

Schreibe eine SPARQL Query, die alle Triples der Daten anzeigt.

In [ ]:
ttl_string = """

@prefix : <https://example.com/>.
@prefix schema: <http://schema.org/>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.

:KarinKeller-Sutter schema:hasOccupation :Bundesrat;
    :partei :FDP;
    :anzahlGeschwister 3.

:ViolaAmherd schema:hasOccupation :Bundesrat;
    :partei :DieMitte;
    :vollerName "Viola Patricia Amherd".

:NadineMasshardt schema:hasOccupation :Nationalrat;
    :partei :SP;
    :geborenIn :AffolternAmAlbis;
    schema:birthDate "1984-10-04"^^xsd:date.

:MayaGraf schema:hasOccupation :Staenderat;
    :partei :Gruene;
    schema:parent :FritzGraf.

:FritzGraf schema:hasOccupation :BaselbieterLandrat;
    :partei :SVP;
    schema:children :MayaGraf.

:FDP rdf:type :Partei.
:DieMitte rdf:type :Partei.
:SP rdf:type :Partei.
:Gruene rdf:type :Partei.
:SVP rdf:type :Partei.

"""

query_string = """

PREFIX : <https://example.com/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?s ?p ?o.

}

"""

plot = parse_and_plot(ttl_string)
display(plot)

df = query(ttl_string, query_string)
display(df)

## SPARQL Query: Alle Parteien

In [ ]:
query_string = """

PREFIX : <https://example.com/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>


SELECT * WHERE {

    ?partei rdf:type :Partei.

}

"""

df = query(ttl_string, query_string)
display(df)